In [1]:
from liveF1Wrapper.adapter import LivetimingF1Adapter, LivetimingF1Request
from liveF1Wrapper.season import Season
from liveF1Wrapper.functions import *
from liveF1Wrapper.events import *
import urllib
import json

In [2]:
def get_season(
    season:int
    ):
    season_data = LivetimingF1Request(urllib.parse.urljoin(str(season) + "/", "Index.json"))
    return Season(**season_data)

In [3]:
s = get_season(2023)

In [12]:
m = s.meetingObjs[1]
help(m)

Help on Meeting in module liveF1Wrapper.weekend object:

class Meeting(builtins.object)
 |  Meeting(**kwargs)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  parse_seasons(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables
 |  
 |  __weakref__
 |      list of weak references to the object



In [11]:
s.meetingObjs[1].sessions[4].name

'Race'

In [13]:
season = 2023
adapter = LivetimingF1Adapter()
endpoint = urllib.parse.urljoin(str(season) + "/", "Index.json")
res_text = adapter.get(endpoint=endpoint)
data = json.loads(res_text)

In [18]:
data

{'Year': 2023,
 'Meetings': [{'Sessions': [{'Key': 9222,
     'Type': 'Practice',
     'Number': 1,
     'Name': 'Practice 1',
     'StartDate': '2023-02-23T10:00:00',
     'EndDate': '2023-02-23T19:30:00',
     'GmtOffset': '03:00:00',
     'Path': '2023/2023-02-25_Pre-Season_Testing/2023-02-23_Practice_1/'},
    {'Key': 7763,
     'Type': 'Practice',
     'Number': 2,
     'Name': 'Practice 2',
     'StartDate': '2023-02-24T10:00:00',
     'EndDate': '2023-02-24T19:30:00',
     'GmtOffset': '03:00:00',
     'Path': '2023/2023-02-25_Pre-Season_Testing/2023-02-24_Practice_2/'},
    {'Key': 7764,
     'Type': 'Practice',
     'Number': 3,
     'Name': 'Practice 3',
     'StartDate': '2023-02-25T10:00:00',
     'EndDate': '2023-02-25T19:30:00',
     'GmtOffset': '03:00:00',
     'Path': '2023/2023-02-25_Pre-Season_Testing/2023-02-25_Practice_3/'}],
   'Key': 1140,
   'Code': 'BRN0104',
   'Number': 1,
   'Location': 'Sakhir',
   'OfficialName': 'FORMULA 1 ARAMCO PRE-SEASON TESTING 2023',

In [17]:
data["Meetings"][0]

{'Sessions': [{'Key': 9222,
   'Type': 'Practice',
   'Number': 1,
   'Name': 'Practice 1',
   'StartDate': '2023-02-23T10:00:00',
   'EndDate': '2023-02-23T19:30:00',
   'GmtOffset': '03:00:00',
   'Path': '2023/2023-02-25_Pre-Season_Testing/2023-02-23_Practice_1/'},
  {'Key': 7763,
   'Type': 'Practice',
   'Number': 2,
   'Name': 'Practice 2',
   'StartDate': '2023-02-24T10:00:00',
   'EndDate': '2023-02-24T19:30:00',
   'GmtOffset': '03:00:00',
   'Path': '2023/2023-02-25_Pre-Season_Testing/2023-02-24_Practice_2/'},
  {'Key': 7764,
   'Type': 'Practice',
   'Number': 3,
   'Name': 'Practice 3',
   'StartDate': '2023-02-25T10:00:00',
   'EndDate': '2023-02-25T19:30:00',
   'GmtOffset': '03:00:00',
   'Path': '2023/2023-02-25_Pre-Season_Testing/2023-02-25_Practice_3/'}],
 'Key': 1140,
 'Code': 'BRN0104',
 'Number': 1,
 'Location': 'Sakhir',
 'OfficialName': 'FORMULA 1 ARAMCO PRE-SEASON TESTING 2023',
 'Name': 'Pre-Season Testing',
 'Country': {'Key': 36, 'Code': 'BRN', 'Name': 'Bahra

In [3]:
season = Season(2024)

In [4]:
print(season)

                                           meeting_code  meeting_key  \
season_year meeting_location  session_type                             
2024        Barcelona         Practice 1        ESP0112         1238   
                              Practice 2        ESP0112         1238   
                              Practice 3        ESP0112         1238   
                              Qualifying        ESP0112         1238   
                              Race              ESP0112         1238   
            Spielberg         Practice 1        AUT0108         1239   
                              Qualifying        AUT0108         1239   
                              Race              AUT0108         1239   
                              Qualifying        AUT0108         1239   
                              Race              AUT0108         1239   
            Silverstone       Practice 1        GBR0114         1240   
                              Practice 2        GBR0114         

TypeError: __str__ returned non-string (type NoneType)

In [2]:
season = 2024
endpoint = urllib.parse.urljoin(str(season) + "/", "Index.json")
LivetimingF1Request(endpoint)

{'Year': 2024,
 'Meetings': [{'Sessions': [{'Key': 9532,
     'Type': 'Practice',
     'Number': 1,
     'Name': 'Practice 1',
     'StartDate': '2024-06-21T13:30:00',
     'EndDate': '2024-06-21T14:30:00',
     'GmtOffset': '02:00:00',
     'Path': '2024/2024-06-23_Spanish_Grand_Prix/2024-06-21_Practice_1/'},
    {'Key': 9533,
     'Type': 'Practice',
     'Number': 2,
     'Name': 'Practice 2',
     'StartDate': '2024-06-21T17:00:00',
     'EndDate': '2024-06-21T18:00:00',
     'GmtOffset': '02:00:00',
     'Path': '2024/2024-06-23_Spanish_Grand_Prix/2024-06-21_Practice_2/'},
    {'Key': 9534,
     'Type': 'Practice',
     'Number': 3,
     'Name': 'Practice 3',
     'StartDate': '2024-06-22T12:30:00',
     'EndDate': '2024-06-22T13:30:00',
     'GmtOffset': '02:00:00',
     'Path': '2024/2024-06-23_Spanish_Grand_Prix/2024-06-22_Practice_3/'},
    {'Key': 9535,
     'Type': 'Qualifying',
     'Name': 'Qualifying',
     'StartDate': '2024-06-22T16:00:00',
     'EndDate': '2024-06-22T1